In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.Connection('data (3).sqlite')

In [2]:
q = """
SELECT lastName, firstName, officeCode
FROM employees
JOIN offices
    USING(officeCode)
WHERE country = "USA"
;"""

pd.read_sql(q, conn)

,lastName,firstName,officeCode
0,Bow,Anthony,1
1,Firrelli,Jeff,1
2,Jennings,Leslie,1
3,Murphy,Diane,1
4,Patterson,Mary,1
5,Thompson,Leslie,1
6,Firrelli,Julie,2
7,Patterson,Steve,2
8,Tseng,Foon Yue,3
9,Vanauf,George,3


In [3]:
#Replacing the JOIN with a nested query

q = """
SELECT lastName, firstName, officeCode
FROM employees
WHERE officeCode IN (SELECT officeCode
                     FROM offices 
                     WHERE country = "USA")
;
"""
pd.read_sql(q, conn)

,lastName,firstName,officeCode
0,Murphy,Diane,1
1,Patterson,Mary,1
2,Firrelli,Jeff,1
3,Bow,Anthony,1
4,Jennings,Leslie,1
5,Thompson,Leslie,1
6,Firrelli,Julie,2
7,Patterson,Steve,2
8,Tseng,Foon Yue,3
9,Vanauf,George,3


In [4]:
# No entiendo

q = """
SELECT lastName, firstName, officeCode
FROM employees
WHERE officeCode IN (
    SELECT officeCode 
    FROM offices 
    JOIN employees
        USING(officeCode)
    GROUP BY 1
    HAVING COUNT(employeeNumber) >= 5
)
;
"""
pd.read_sql(q, conn)

,lastName,firstName,officeCode
0,Murphy,Diane,1
1,Patterson,Mary,1
2,Firrelli,Jeff,1
3,Bondur,Gerard,4
4,Bow,Anthony,1
5,Jennings,Leslie,1
6,Thompson,Leslie,1
7,Bondur,Loui,4
8,Hernandez,Gerard,4
9,Castillo,Pamela,4


In [5]:
# Average per customer of the average of each payment
#Take all the payment orders for ONE customer and calculate the average,
# Then take all the average per customer and take the average again 

q = """
SELECT AVG(customerAvgPayment) AS averagePayment
FROM (
    SELECT AVG(amount) AS customerAvgPayment
    FROM payments
    JOIN customers
        USING(customerNumber)
    GROUP BY customerNumber
)
;"""
pd.read_sql(q, conn)

,averagePayment
0,31489.754582


In [8]:
# Foreign Key
#  Both Keys conneted have differnt names

q = """
SELECT lastName, firstName, employeeNumber
FROM employees
WHERE employeeNumber IN (SELECT salesRepEmployeeNumber
                     FROM customers 
                     WHERE country = "USA")
;
"""
pd.read_sql(q, conn)

,lastName,firstName,employeeNumber
0,Jennings,Leslie,1165
1,Thompson,Leslie,1166
2,Firrelli,Julie,1188
3,Patterson,Steve,1216
4,Tseng,Foon Yue,1286
5,Vanauf,George,1323


Step 10
Having looked at the results from above, the product team is curious to dig into the underperforming products. They want to ask members of the team who have sold these products about what kind of messaging was successful in getting a customer to buy these specific products. Using a subquery or common table expression (CTE), select the employee number, first name, last name, city of the office, and the office code for employees who sold products that have been ordered by fewer than 20 customers.

Hint: Start with the subquery, find all the products that have been ordered by 19 or less customers, consider adapting one of your previous queries.

Selection deleted


In [9]:
# WRONG
# Replace None with your code
df_under_20 = pd.read_sql("""
                SELECT employeeNumber, firstName, lastName, o.city, officeCode, productName,
                    COUNT(DISTINCT customerNumber) AS num_customers                     
                FROM employees AS e
                JOIN offices o
                    USING(officeCode)
                JOIN customers AS c
                    ON employeeNumber = c.salesRepEmployeeNumber
                JOIN orders
                    USING(customerNumber)    
                JOIN orderdetails
                    USING(orderNumber)
                JOIN products AS p
                    USING(productCode)   
                GROUP BY e.employeeNumber, e.firstName, e.lastName, o.city, o.officeCode, p.productName                               
                HAVING num_customers < 20
;""", conn)

df_under_20

,employeeNumber,firstName,lastName,city,officeCode,productName,num_customers
0,1165,Leslie,Jennings,San Francisco,1,18th Century Vintage Horse Carriage,4
1,1165,Leslie,Jennings,San Francisco,1,18th century schooner,2
2,1165,Leslie,Jennings,San Francisco,1,1900s Vintage Bi-Plane,1
3,1165,Leslie,Jennings,San Francisco,1,1900s Vintage Tri-Plane,2
4,1165,Leslie,Jennings,San Francisco,1,1903 Ford Model A,1
...,...,...,...,...,...,...,...
1363,1702,Martin,Gerard,Paris,4,The Mayflower,1
1364,1702,Martin,Gerard,Paris,4,The Queen Mary,2
1365,1702,Martin,Gerard,Paris,4,The Schooner Bluenose,2
1366,1702,Martin,Gerard,Paris,4,The Titanic,2


In [10]:
# Right

pd.read_sql("""
    SELECT 
        e.employeeNumber, 
        e.firstName, 
        e.lastName, 
        o.city, 
        o.officeCode, 
        p.productName,
        COUNT(DISTINCT c.customerNumber) AS num_customers
    FROM employees AS e
    JOIN offices o ON e.officeCode = o.officeCode
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders ord ON c.customerNumber = ord.customerNumber
    JOIN orderdetails od ON ord.orderNumber = od.orderNumber
    JOIN products p ON od.productCode = p.productCode
    WHERE p.productCode IN (
        SELECT od2.productCode
        FROM orderdetails od2
        JOIN orders o2 ON od2.orderNumber = o2.orderNumber
        JOIN customers c2 ON o2.customerNumber = c2.customerNumber
        GROUP BY od2.productCode
        HAVING COUNT(DISTINCT c2.customerNumber) < 20
    )
    GROUP BY 
        e.employeeNumber, 
        e.firstName, 
        e.lastName, 
        o.city, 
        o.officeCode, 
        p.productName
    ORDER BY e.employeeNumber;
""", conn)


,employeeNumber,firstName,lastName,city,officeCode,productName,num_customers
0,1165,Leslie,Jennings,San Francisco,1,1949 Jaguar XK 120,2
1,1165,Leslie,Jennings,San Francisco,1,1952 Citroen-15CV,2
2,1165,Leslie,Jennings,San Francisco,1,1958 Chevy Corvette Limited Edition,2
3,1165,Leslie,Jennings,San Francisco,1,1969 Chevrolet Camaro Z28,2
4,1165,Leslie,Jennings,San Francisco,1,2002 Chevy Corvette,2
5,1166,Leslie,Thompson,San Francisco,1,1949 Jaguar XK 120,1
6,1166,Leslie,Thompson,San Francisco,1,1952 Citroen-15CV,1
7,1166,Leslie,Thompson,San Francisco,1,1969 Chevrolet Camaro Z28,1
8,1166,Leslie,Thompson,San Francisco,1,2002 Chevy Corvette,2
9,1188,Julie,Firrelli,Boston,2,1958 Chevy Corvette Limited Edition,1


In [11]:
pd.read_sql("""
    SELECT DISTINCT 
        e.employeeNumber,
        e.firstName,
        e.lastName,
        o.city,
        o.officeCode,
        p.productName,
        low_perf.num_customers
    FROM employees e
    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders ord ON c.customerNumber = ord.customerNumber
    JOIN orderdetails od ON ord.orderNumber = od.orderNumber
    JOIN products p ON od.productCode = p.productCode
    JOIN offices o ON e.officeCode = o.officeCode
    JOIN (
        SELECT 
            od2.productCode,
            COUNT(DISTINCT c2.customerNumber) AS num_customers
        FROM orderdetails od2
        JOIN orders o2 ON od2.orderNumber = o2.orderNumber
        JOIN customers c2 ON o2.customerNumber = c2.customerNumber
        GROUP BY od2.productCode
        HAVING COUNT(DISTINCT c2.customerNumber) < 20
    ) AS low_perf ON od.productCode = low_perf.productCode
    ORDER BY e.employeeNumber;
""", conn)


,employeeNumber,firstName,lastName,city,officeCode,productName,num_customers
0,1165,Leslie,Jennings,San Francisco,1,1949 Jaguar XK 120,18
1,1165,Leslie,Jennings,San Francisco,1,1958 Chevy Corvette Limited Edition,19
2,1165,Leslie,Jennings,San Francisco,1,1952 Citroen-15CV,18
3,1165,Leslie,Jennings,San Francisco,1,1969 Chevrolet Camaro Z28,18
4,1165,Leslie,Jennings,San Francisco,1,2002 Chevy Corvette,18
5,1166,Leslie,Thompson,San Francisco,1,1949 Jaguar XK 120,18
6,1166,Leslie,Thompson,San Francisco,1,1952 Citroen-15CV,18
7,1166,Leslie,Thompson,San Francisco,1,1969 Chevrolet Camaro Z28,18
8,1166,Leslie,Thompson,San Francisco,1,2002 Chevy Corvette,18
9,1188,Julie,Firrelli,Boston,2,1958 Chevy Corvette Limited Edition,19
